#### WebAssembly vs RISC-V

As a reminder, here are some WebAssembly instructions:

| instruction           | effect                                             | trap condition   |
|:----------------------|:---------------------------------------------------|:-----------------|
| `i32.const i`         | `s[sp], sp := i, sp + 1`                           | `sp < StackSize` |
| `i32.add`             | `s[sp - 2], sp := s[sp - 2] + s[sp - 1], sp - 1`   |                  |
| `i32.sub`             | `s[sp - 2], sp := s[sp - 2] - s[sp - 1], sp - 1`   |                  |
| `i32.mul`             | `s[sp - 2], sp := s[sp - 2] × s[sp - 1], sp - 1`   |                  |
| `i32.div_s`           | `s[sp - 2], sp := s[sp - 2] div s[sp - 1], sp - 1` | `s[sp - 1] = 0`  |
| `i32.rem_s`           | `s[sp - 2], sp := s[sp - 2] mod s[sp - 1], sp - 1` | `s[sp - 1] = 0`  |
| `i32.eqz`             | `s[sp - 1] := s[sp - 1] = 0`                       |                  |
| `i32.eq`              | `s[sp - 2], sp := s[sp - 2] = s[sp - 1], sp - 1`   |                  |
| `i32.ne`              | `s[sp - 2], sp := s[sp - 2] ≠ s[sp - 1], sp - 1`   |                  |
| `i32.lt_s`            | `s[sp - 2], sp := s[sp - 2] < s[sp - 1], sp - 1`   |                  |
| `i32.gt_s`            | `s[sp - 2], sp := s[sp - 2] > s[sp - 1], sp - 1`   |                  |
| `i32.le_s`            | `s[sp - 2], sp := s[sp - 2] ≤ s[sp - 1], sp - 1`   |                  |
| `i32.ge_s`            | `s[sp - 2], sp := s[sp - 2] ≥ s[sp - 1], sp - 1`   |                  |
| `i32.load offset = n` | `s[sp - 1] := m[s[sp - 1] + n]`                    | `0 ≤ s[sp - 1] + n < MemSize` |
| `i32.store offset = n`| `m[s[sp - 2] + n], sp := s[sp - 1], sp - 2`        | `0 ≤ s[sp - 1] + n < MemSize` |
| `local.get x`         | `s[sp], sp := s[loc(x)], sp + 1`                   |                  |
| `local.set x`         | `s[loc(x)], sp := s[sp - 1], sp - 1`               |                  |
| `global.get x`        | `s[sp], sp := g[loc(x)], sp + 1`                   |                  |
| `global.set x`        | `g[loc(x)], sp := s[sp - 1], sp - 1`               |                  |

Here are some RISC-V instructions:

| Instruction | Effect| Description| 
|:----|:----|:----|
|add rd, rs1, rs2 | R[rd], pc := (𝙪R[rs1] + 𝙪R[rs2]) 𝗺𝗼𝗱 2³², pc + 4| addition⁽¹⁾ |
|sub rd, rs1, rs2 | R[rd], pc := (𝙪R[rs1] – 𝙪R[rs2]) 𝗺𝗼𝗱 2³², pc + 4 | subtraction⁽¹⁾|
|and rd, rs1, rs2 | R[rd], pc := R[rs1] & R[rs2], pc + 4 | bitwise and⁽²⁾ |
|or rd, rs1, rs2 | R[rd], pc := R[rs1] \| R[rs2], pc + 4 | bitwise or⁽³⁾ |
|xor rd, rs1, rs2 | R[rd], pc := R[rs1] ^ R[rs2], pc + 4 | bitwise xor⁽⁴⁾ |
|sll rd, rs1, rs2 | R[rd], pc := R[rs1] 𝘀𝗹𝗹 𝙪R[rs2], pc + 4 | shift left⁽⁵⁾ |
|sra rd, rs1, rs2 | R[rd], pc := 1 if R[rs1] 𝘀𝗿𝗮 𝙪R[rs2]) else 0, pc + 4 | shift right arithmetic⁽⁶⁾ |
|srl rd, rs1, rs2 | R[rd], pc := 1 if R[rs1] 𝘀𝗿𝗹 𝙪R[rs2]) else 0, pc + 4 | shift right (logical)⁽⁷⁾ |
|slt rd, rs1, rs2 | R[rd], pc := 1 if 𝙨R[rs1]) < 𝙨R[rs2]) else 0, pc + 4 | signed comparison |
|sltu rd, rs1, rs2 | R[rd], pc := 1 if 𝙪R[rs1] < 𝙪R[rs2] else 0, pc + 4 | unsigned comparison |

1) Addition and subtraction interpret registers as unsigned integers. The result is also correct if the registers hold signed integers in two's complement and the result fits in a word.
2) For words `a` and `b`: `(a & b)[i] = 1 𝗶𝗳 a[i] = 1 ∧ b[i] = 1 𝗲𝗹𝘀𝗲 0`, for `0 ≤ i < 32`.
3) For words `a` and `b`: `(a | b)[i] = 1 𝗶𝗳 a[i] = 1 ∨ b[i] = 1 𝗲𝗹𝘀𝗲 0`, for `0 ≤ i < 32`.
4) For words `a` and `b`: `(a ^ b)[i] = 1 𝗶𝗳 a[i] ≠ b[i] 𝗲𝗹𝘀𝗲 0`, for `0 ≤ i < 32`.
5) For word `a`: `(a 𝘀𝗹𝗹 n)[i] = a[i – n]`, for `n ≤ i < 32` and `(a 𝘀𝗹𝗹 n)[i] = 0` for `0 ≤ i < n` and for `0 ≤ n < 32`.
6) For word `a`: `(a 𝘀𝗿𝗮 n)[i] = a[i + n]`, for `0 ≤ i < 32 – n` and `(a 𝘀𝗿𝗮 n)[i] = a[31]` for `32 – n ≤ i < 32` and for `0 ≤ n < 32`.
7) For word `a`: `(a 𝘀𝗿𝗹 n)[i] = a[i + n]`, for `0 ≤ i < 32 – n` and `(a 𝘀𝗿𝗹 n)[i] = 0` for `32 – n ≤ i < 32` and for `0 ≤ n < 32`.

The characteristics of WebAssembly are:
- Stack architecture
- Byte code
- Statically typed
- Block-structured
- Non-uniform store

For each of these:
1. explain briefly,
2. give one or more instructions to illustrate your explanation,
3. discuss if and how RISC-V (or any other modern architecture) differs!

 - Stack architecture  
WebAssembly pushes value onto stack to perform various operations. For example, when we are trying to add values `1` and `2`, we would use `i32.const 1` follow by `i32.const 2` to push values `1` and `2` onto the stack. Then, we would use `i32.add` that pops the top `2` items from the stack to perform the addition. After calling `i32.add`, the result `3` is also pushed onto the stack. On ther other hand, RISC-V uses registers to perform various operation instead of using stack

- Byte code
In WebAssembly, allocation of memory does not need to be word aligned. 

- Statically typed
Variable types in WebAssembly is statically types, meaning user would have to define the type of the variable upon the variable declaration. For example, function parameters are typed with type index from the types table. On the other hand, variables in RISC-V can be dynamically type, meaning user do not have to define the type of a variable upon declaration, the type of the variable is determined during runtime.
  
- Block-structured 
WebAssembly uses block to define different sections of the code, allowing code to jump to different blocks by using `br`. Whereas in RISC-V, different sections of the code are defined by labels, users can jump to different blocks by using the label name and the `jl` instruction.

- Non-uniform store  
For storing values, WebAssembly uses various form of stores, it uses global store, stack and the main memory. When we want to set a value to the global variable, we would perform `global.set x`. When we want to push parameters to invoke a function, we would push parameter values to the stack (please refer to the example from point "Stack architecture"). When we need to declare a global or local array or record, we would use the main memory. In contrast, RISC-V only uses registers and the main memory to store values.